In [64]:
import pandas as pd
import numpy as np
import requests
import io
import datetime

In [65]:
tickers = ["AEP", "DFSVX", "DFLVX", "FSAGX"]
GS1_URL = "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1318&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=GS1&scale=left&cosd=1953-04-01&coed=2024-02-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2024-03-11&revision_date=2024-03-11&nd=1953-04-01"

START_DATE = "01/01/1995"
END_DATE = "09/01/2023"

In [66]:
from yahoo_fin.stock_info import get_data

tickers_data = {}
for t in tickers:
    tickers_data[t] = get_data(t, start_date=START_DATE, end_date=END_DATE, index_as_date=False, interval="1mo")

In [67]:

gs1_data = requests.get(GS1_URL)
fred = pd.read_csv(io.StringIO(gs1_data.text), parse_dates=["DATE"])
fred = fred.rename(columns={"DATE":"date"})
fred = fred.loc[(fred['date'] >= START_DATE) & (fred['date'] < END_DATE)]

'''
for t in tickers:
    df = pd.read_json(f"./data/{t}.json", convert_dates=["Date"])
    merged = pd.merge(merged, df[["date", "Adj Close"]], on="Date", how="inner")
    merged = merged.rename(columns={"Adj Close":f"{t}_Adjusted"})
'''

print(fred.head())

          date   GS1
501 1995-01-01  7.05
502 1995-02-01  6.70
503 1995-03-01  6.43
504 1995-04-01  6.27
505 1995-05-01  6.00


In [68]:
merged = fred
for t in tickers_data:
    df = tickers_data[t]
    merged = pd.merge(merged, df[["date", "adjclose"]], on="date", how="inner")
    merged = merged.rename(columns={"adjclose":f"{t}_Adjusted"})

In [69]:
shifted = merged.shift(12)
for t in tickers:
    col_to_substract = f"{t}_Adjusted"
    merged[f"{t}_annual"] = (merged[col_to_substract]- shifted[col_to_substract]) / shifted[col_to_substract]
    merged[f"{t}_excess"] = merged[f"{t}_annual"] - merged.shift(12)["GS1"]/100

print(merged.shift(-12).head())

        date   GS1  AEP_Adjusted  DFSVX_Adjusted  DFLVX_Adjusted  \
0 1996-01-01  5.09     12.219301        2.448707        3.688002   
1 1996-02-01  4.94     11.839608        2.502315        3.741763   
2 1996-03-01  5.34     11.688332        2.571487        3.857351   
3 1996-04-01  5.54     11.373378        2.704645        3.935062   
4 1996-05-01  5.64     11.233397        2.810134        3.999795   

   FSAGX_Adjusted  AEP_annual  AEP_excess  DFSVX_annual  DFSVX_excess  \
0       11.230024    0.354032    0.283532      0.275800      0.205300   
1       11.678019    0.355528    0.288528      0.259310      0.192310   
2       11.958012    0.403812    0.339512      0.278871      0.214571   
3       12.242316    0.324274    0.261574      0.299164      0.236464   
4       13.577686    0.250692    0.190692      0.313527      0.253527   

   DFLVX_annual  DFLVX_excess  FSAGX_annual  FSAGX_excess  
0      0.391165      0.320665      0.462143      0.391643  
1      0.335715      0.268715   

In [70]:
return_dist = {}
for t in tickers:
    cleaned = merged[merged[f"{t}_excess"].notna()]
    return_dist[t] = (cleaned[f"{t}_excess"].mean(), cleaned[f"{t}_excess"].std())

print(return_dist)

{'AEP': (0.07754664812650876, 0.19979382774849208), 'DFSVX': (0.1094686494540554, 0.24398203147580538), 'DFLVX': (0.08998956240686244, 0.19464668440220523), 'FSAGX': (0.052988783083607105, 0.3088840978004022)}


In [71]:
## Calulating correlation matrix
cols = [f"{col}_excess" for col in tickers]
rename_dict = {}

for col in cols:
    rename_dict[col] = col.replace("_excess","")
    
corr = merged[cols].corr()
corr = corr.rename(index=rename_dict, columns=rename_dict)
print(corr)

            AEP     DFSVX     DFLVX     FSAGX
AEP    1.000000  0.393708  0.561141 -0.035950
DFSVX  0.393708  1.000000  0.902942  0.091180
DFLVX  0.561141  0.902942  1.000000  0.026616
FSAGX -0.035950  0.091180  0.026616  1.000000


In [72]:
## Calulating variance-covariance
var_covar = corr.copy()
for row_label, row_data in var_covar.iterrows():
    row_std = return_dist[row_label][1]
    for col_label in var_covar.columns:
        col_std = return_dist[col_label][1]
        row_data[col_label] = row_data[col_label]*row_std*col_std

print(var_covar)

            AEP     DFSVX     DFLVX     FSAGX
AEP    0.039918  0.019192  0.021822 -0.002219
DFSVX  0.019192  0.059527  0.042881  0.006872
DFLVX  0.021822  0.042881  0.037887  0.001600
FSAGX -0.002219  0.006872  0.001600  0.095409


In [73]:
weights = np.array(
    [[0.36656],
    [0.32678],
    [0.15174],
    [0.15492]]
    )


res = weights.T @ np.power(var_covar.to_numpy() @ weights,0.5)
sd = res[0,0]

risk_prem = np.dot(weights.T, np.array([[return_dist[r][0]] for r in return_dist.keys()]))[0,0]
sharpie_ratio  = risk_prem/sd


In [74]:
# TODO: need to figure out the arbitrary values

price_of_risk = risk_prem/sd**2
risk_aver = 4
y = price_of_risk/risk_aver

DGS1 = 5.37/100
E_rf = (1-y)*DGS1
E_rP = y * (DGS1+risk_prem)

expected_return =  E_rf+E_rP
riskiness = y * sd